<a href="https://colab.research.google.com/github/Saadman53/Thesis-Human-Activity-Recognition/blob/Defense/Feature_Extraction_using_Alexnet(MEI_Prewitt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
gdrive_path = "drive/My Drive/Dataset/MEI_Prewitt_227x227.mat"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data

import torchvision
import torchvision.transforms as transforms



import random
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report


from torch.utils.data import DataLoader, ConcatDataset
# from torchvision import transforms
from sklearn.model_selection  import StratifiedKFold

import torch.optim.lr_scheduler as lr_scheduler
%matplotlib inline

import scipy.io as sio
torch.autograd.set_detect_anomaly(True)


from functools import partial
from typing import Any, Optional

import torch
import torch.nn as nn


In [3]:
dat = sio.loadmat(gdrive_path)

In [4]:
dat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data', 'filename', 'label'])

In [5]:
X = dat['data']
y = dat['label'].ravel()

In [6]:
X.shape

(1165, 227, 227)

In [7]:
def alexnet_feature_extraction(X):
  'Instantiate the model'
  alexnet = torchvision.models.alexnet(pretrained=True)
  alexnet.classifier = torch.nn.Sequential(*list(alexnet.classifier.children())[:-4]) #keeping only the first fc layer

  'Necessary transformations for Alexnet'
  data_transforms = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize((256),interpolation=transforms.InterpolationMode.BILINEAR),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
  ])

  'Convert to 3 channel'
  dataset = torch.empty((X.shape[0],4096),dtype=torch.float32)
  for i in range(X.shape[0]):
    t = torch.tensor(X[i], dtype = torch.float32).reshape((1,227,227))
    t = t.expand(3,*t.shape[1:])
    dataset[i] = alexnet(torch.unsqueeze(data_transforms(t),0)).ravel()
    print(i*100/X.shape[0])
  dataset = dataset.detach().numpy()
  return dataset

In [8]:
data = alexnet_feature_extraction(X)

0.0
0.08583690987124463
0.17167381974248927
0.2575107296137339
0.34334763948497854
0.4291845493562232
0.5150214592274678
0.6008583690987125
0.6866952789699571
0.7725321888412017
0.8583690987124464
0.944206008583691
1.0300429184549356
1.1158798283261802
1.201716738197425
1.2875536480686696
1.3733905579399142
1.4592274678111588
1.5450643776824033
1.6309012875536482
1.7167381974248928
1.8025751072961373
1.888412017167382
1.9742489270386265
2.060085836909871
2.1459227467811157
2.2317596566523603
2.3175965665236054
2.40343347639485
2.4892703862660945
2.575107296137339
2.6609442060085837
2.7467811158798283
2.832618025751073
2.9184549356223175
3.004291845493562
3.0901287553648067
3.1759656652360513
3.2618025751072963
3.347639484978541
3.4334763948497855
3.51931330472103
3.6051502145922747
3.6909871244635193
3.776824034334764
3.8626609442060085
3.948497854077253
4.034334763948498
4.120171673819742
4.206008583690987
4.291845493562231
4.377682403433476
4.463519313304721
4.549356223175966
4.63519

In [9]:
print(data.shape)

(1165, 4096)


In [11]:
dic = {'data': data, 'labels': y, 'filename': dat['filename'].ravel()}
sio.savemat("drive/My Drive/Dataset/Final Dataset/alexnet_extracted_MEI_Prewitt_4096.mat", dic)